### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import os
import json
import requests
from tqdm import tqdm
import time
import keras

C:\Users\riskf\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riskf\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\riskf\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def build_encoder(latent_dim=20):
    encoder_inputs = layers.Input(shape=(100,)) #change here features*2
    x = layers.Dense(256, activation='relu')(encoder_inputs)
    x = layers.Dense(64, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    return encoder

def build_decoder(latent_dim=20):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(64, activation='relu')(latent_inputs)
    x = layers.Dense(256, activation='relu')(x)
    decoder_outputs = layers.Dense(100, activation='linear')(x) #change here features*2
    decoder = models.Model(latent_inputs, decoder_outputs, name="decoder")
    return decoder

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        # Initialize trackers for monitoring losses
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        # Return list of metrics to be updated during training
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs, training=False):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Compute KL divergence loss even during inference to track loss correctly
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        # Only add KL loss during training
        if training:
            self.add_loss(kl_loss)
        return reconstructed

    def train_step(self, data):
    # Unpack the data
        x = data[0] if isinstance(data, tuple) else data

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x, training=True)
            reconstruction = self.decoder(z, training=True)

            # If  data is flat (e.g., shape=(batch_size, features)), adjust axis accordingly
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(x, reconstruction), axis=-1
            )
            reconstruction_loss = tf.reduce_sum(reconstruction_loss)  # Sum over all dimensions

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        return {'loss': total_loss, 'reconstruction_loss': reconstruction_loss, 'kl_loss': kl_loss}

# Example usage:
latent_dim = 16  
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)
vae = VAE(encoder, decoder)
#vae.compile(optimizer='adam')
vae.compile(optimizer=tf.keras.optimizers.Adam())


In [3]:
#relative paths. # Set directory paths for later use.
# Get the directory of the script file
base_dir = os.getcwd()
base_dir
ligants_type=['enzyme','GPCR','ion_channel','nuclear_receptor']
ltype=ligants_type[2]
file_name='final_new_par_NNMF_50.csv'
file_path = os.path.join(base_dir,'data','split',ltype, file_name)
output_path = file_path
data_frame = pd.read_csv(file_path, header=None, skiprows=1)


In [4]:
# Filter to only include instances with label 1 (interactions)
# Separate features and labels
filtered_df = data_frame[data_frame.iloc[:, -1] == 1]  # All rows, all columns except the last one
features_new = filtered_df.iloc[:, :-1]     # All rows, just the last column

# Convert features DataFrame to a NumPy array if necessary
x_train = features_new.to_numpy()

In [5]:
filtered_df.shape

(1476, 101)

In [6]:
#para
epochs=4
batch_size=82

In [7]:
# Train the model
vae.fit(x_train, epochs=epochs, batch_size=batch_size)

Epoch 1/4
18/18 [==============================] - 1s 2ms/step - loss: 0.5379 - reconstruction_loss: 0.4961 - kl_loss: 0.0114
Epoch 2/4
18/18 [==============================] - 0s 1ms/step - loss: 0.4216 - reconstruction_loss: 0.3904 - kl_loss: 0.0116
Epoch 3/4
18/18 [==============================] - 0s 1ms/step - loss: 0.3856 - reconstruction_loss: 0.3555 - kl_loss: 0.0290
Epoch 4/4
18/18 [==============================] - 0s 1ms/step - loss: 0.3751 - reconstruction_loss: 0.3529 - kl_loss: 0.0258


In [8]:
# Filter to only include instances with label 1 (interactions)
filtered_df = data_frame[data_frame.iloc[:, -1] == 1]

# Separate features
features_new = filtered_df.iloc[:, :-1]  # Assuming the last column is the label

In [9]:
x_new = features_new.to_numpy()

In [10]:
z_mean, z_log_var, z = vae.encoder.predict(x_new)
# Now, z contains the latent representations of  filtered data.

47/47 [==============================] - 0s 716us/step


In [11]:
reconstructed_new = vae.predict(x_new)

47/47 [==============================] - 0s 731us/step


In [12]:
# Convert the reconstructed data to a DataFrame
reconstructed_df = pd.DataFrame(reconstructed_new)

# Display the first few rows of the reconstructed DataFrame
print(pd.DataFrame(x_new))
print(reconstructed_df)

            0    1    2    3    4    5         6    7    8    9   ...   90  \
0     0.000001  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
1     0.000001  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
2     0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
3     0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
4     0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
...        ...  ...  ...  ...  ...  ...       ...  ...  ...  ...  ...  ...   
1471  0.000000  0.0  0.0  0.0  0.0  0.0  0.000097  0.0  0.0  0.0  ...  0.0   
1472  0.000000  0.0  0.0  0.0  0.0  0.0  0.000097  0.0  0.0  0.0  ...  0.0   
1473  0.000000  0.0  0.0  0.0  0.0  0.0  0.000097  0.0  0.0  0.0  ...  0.0   
1474  0.000001  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   
1475  0.000001  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0   

       91        92   93        94   95   96        97        9

In [13]:
# Add a new column 'Label' with all values set to 1
reconstructed_df[100] = 1

# Display the first few rows to verify the new column
print(reconstructed_df.head())

        0         1         2         3         4         5         6    \
0 -0.103633  0.537927  0.307721 -0.524982 -0.573096 -0.842427 -0.470250   
1 -0.221411  0.571787  0.518317 -0.339802 -0.444134 -0.479660 -0.444461   
2  0.072201  0.239301  0.284505 -0.576310 -0.368644 -0.660404 -0.403455   
3  0.280037  0.391957  0.286961 -0.450765 -0.743511 -1.079902 -0.553522   
4  0.226580  0.482498  0.366701 -0.269267 -1.203532 -1.215584 -1.121308   

        7         8         9    ...       91        92        93        94   \
0 -0.368201  0.164712  0.119853  ... -0.636139  0.483447 -0.546807  0.110442   
1 -0.037358  0.318593  0.210779  ... -0.315443  0.307611 -0.362484  0.332685   
2 -0.369000  0.244358  0.364121  ... -0.349347  0.097012 -0.320928  0.137036   
3 -0.066083  0.213450  0.067472  ... -0.365850  0.218065 -0.366467 -0.137946   
4 -0.807296  0.581099  0.222833  ... -0.700005  0.304258 -0.855834  0.429490   

        95        96        97        98        99   100  
0 -0.5846

In [14]:
num_samples_to_generate = 39888  # The number of new rows want to generate

# Generate random samples from the latent space
latent_dim = 16  # Ensure this matches the latent dimension size of  VAE
z_new_samples = np.random.normal(size=(num_samples_to_generate, latent_dim))

# Use the decoder to generate new data
new_data_generated = vae.decoder.predict(z_new_samples)



1247/1247 [==============================] - 1s 557us/step


In [15]:
new_data_generated

array([[ 0.69774914,  0.4741873 ,  0.10174187, ..., -0.4961947 ,
        -0.52582306, -0.3121771 ],
       [ 0.00816039,  0.19777365,  0.3355031 , ..., -0.5325238 ,
        -0.4979144 , -0.5786872 ],
       [-0.02794106,  0.18283619,  0.5500798 , ..., -0.6530952 ,
        -0.7283457 , -0.29689863],
       ...,
       [ 0.5801007 ,  0.5647589 ,  0.37657687, ..., -0.7875252 ,
        -0.6218484 , -0.62546796],
       [ 0.0514942 ,  0.16818334,  0.3657479 , ..., -0.3961527 ,
        -0.3736639 , -0.44073653],
       [-0.07328035,  0.5365022 ,  0.25343615, ..., -0.24855587,
        -0.43090078, -0.6993154 ]], dtype=float32)

In [16]:
# Convert the generated data to a DataFrame
new_data_df = pd.DataFrame(new_data_generated)

# Add a column 'Label' with all values set to 1
new_data_df[100] = 1

In [17]:
enhanced_df = pd.concat([data_frame, new_data_df], axis=0).reset_index(drop=True)

In [18]:
enhanced_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.610382,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.030178,0.000000,0.000000,0.000000,0.0
4,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82723,0.350601,0.554283,0.383642,-0.390014,-0.595315,-0.673903,-0.559212,-0.772193,0.272257,0.180662,...,-0.427087,0.342019,-0.447427,0.042554,-0.354853,-0.650851,-0.439533,-0.552443,-0.362309,1.0
82724,0.639235,0.854969,0.640124,-0.730812,-0.648968,-0.908939,-0.655599,-0.752058,0.164801,0.018587,...,-0.603367,0.858650,-0.557059,0.328288,-0.458486,-0.987577,-1.172547,-1.060778,0.002433,1.0
82725,0.580101,0.564759,0.376577,-0.527779,-0.282143,-1.135035,-0.318924,-1.017934,0.045957,-0.113918,...,-0.335162,0.410498,-0.915534,0.019921,-0.490301,-0.595338,-0.787525,-0.621848,-0.625468,1.0
82726,0.051494,0.168183,0.365748,-0.122101,-0.359340,-0.939162,-0.619560,-0.520777,0.553453,0.257805,...,-0.571778,0.159481,-0.445224,0.223576,-0.461257,-0.577598,-0.396153,-0.373664,-0.440737,1.0


In [19]:
file_name='enhanced_VAE_final_new_par_50_NNMF_space_2.csv'
file_path = os.path.join(base_dir,'data','split',ltype, file_name)
output_path = file_path
enhanced_df.to_csv(output_path, index=False)